In [1]:
import xlrd
import os.path
import random
from deap import tools
import numpy as np
import pandas as pd
import math
import random
from collections import defaultdict
import time

/anaconda3/lib/python3.7/site-packages/deap/tools/_hypervolume/pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)
/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


In [2]:
df = pd.read_csv(os.path.join('/Users/firzoknadeem/Desktop/University Files/AI/Project','Test Case 1.csv') , header=None)

In [3]:
df

,0,1,2,3,4,5,6
0,Total Classes in a week,414,NaN,NaN,NaN,NaN,NaN
1,Teacher Count,46,NaN,NaN,NaN,NaN,NaN
2,Total Rooms,15,NaN,NaN,NaN,NaN,NaN
3,Groups Count,23,NaN,NaN,NaN,NaN,NaN
4,Allocations,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,C1,C2,C3,C4,C5,C6
6,BatchCS16_A,T1,T2,T3,T4,T5,T6
7,class duration,1,1.5,1,1.5,1,1
8,BatchCS16_B,T1,T2,T3,T4,T5,T6
9,class duration,1,1.5,1,1.5,1,1


In [4]:
class gene:
    def __init__(self, day, room, time):
        self.day = day
        self.room = room
        self.time = float(time)

In [5]:
class slot:#BatchCS16_A$C1$T1$1
    def __init__(self, group, course, teacher, duration):
        self.group = group
        self.course = course
        self.teacher = teacher
        self.duration = float(duration)

In [6]:
totalColumns = df.shape[1]

In [7]:
#searching for the starting of the row data
rowStart = 0
courseStart='C1'
found = False
for i in range(0, df.shape[0]):
    for j in range(df.shape[1]):
        val = df.values[i][j]
        if val == courseStart:
            found = True
            break
    if found:
        break
rowStart = i
print ("rowStart", i) #---> sample data starting from this row

#getting the courses names
courses = []
noCourses = totalColumns-1
for k in range(totalColumns):
    val = df.values[rowStart, k]
    if val == val:#Check for nan
        courses.append(val)
print ("courses", courses)
rowStart += 1
print ("rowStart", rowStart) #---> sample data starting from this row

rowStart 5
courses ['C1', 'C2', 'C3', 'C4', 'C5', 'C6']
rowStart 6


In [8]:
#finding exact number of rows that have the value i.e. data of corresponding teacher and section
totalRows = 0 #----> has the total number of rows that contains data
rowList = [] #contains the integer values of rows
p = 'Preferences '
found = False
for i in range(rowStart, df.shape[0]):
    for j in range(df.shape[1]):
        val = df.values[i][j]
        if val == p:
            found = True
    if found:
        break
    rowList.append(i)

totalRows = len(rowList)
print ("totalRows", totalRows)
print ("rowList", rowList)

totalRows 46
rowList [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51]


In [9]:
totalClassesInWeek = int(df.values[0, 1])

teacherCount = int(df.values[1, 1])

roomCount = int(df.values[2, 1])

groupCount = int(df.values[3, 1])

In [10]:
#Getting all teachers preferences in tuples

startPreferenceIndex = df.loc[df[0] == 'Preferences '].index[0] + 1    # getting the starting index of teacher's preference
teacherPreference = {}
for i in range(startPreferenceIndex,df.shape[0]):
    teacherPreference[df.values[i,0]] = df.values[i,1]

In [11]:
#hashtable for chromosomes
#all possible classes for groups and courses
slots = []
for i in range(rowStart, rowStart + totalRows, 2):

    for j, c in enumerate(courses):
        val = slot(str(df.values[i,0].strip()), str(c), str(df.values[i, j+1].strip()), str(df.values[i+1, j+1]))
#         val=str(df.values[i,0].strip())#Group e.g BatchCS16_A
#         val+="$"
#         val+=str(c)#courses e.g C1
#         val+="$"
#         val+=str(df.values[i, j+1].strip())#teacher e.g T1
#         val+="$"
#         val+=str(df.values[i+1, j+1])#duration
        slots.append(val)
        slots.append(val)
        
        if float(df.values[i+1, j+1])==1:
            slots.append(val)
        

In [12]:
print(len(slots))
print(slots[0])

381


In [13]:
#Possible values for chromosomes e.g Mon_1_8.5

days = ["Mon", "Tue", "Wed", "Thu", "Fri"]
rooms = list(range(1, roomCount+1))
startTime = np.arange(8.5, 17, 0.5)

day_room_time = []
for d in days:
    for r in rooms:
        for s in startTime:
            
            val = gene(day=d, room=r, time=s)
            
#             val=""
#             val+=(str(d))
#             val+=("_")
#             val+=(str(r))
#             val+=("_")
#             val+=(str(s))
            day_room_time.append(val)

In [14]:
print(len(day_room_time))
print(day_room_time[0])

1275


In [15]:
#First population
#Now randomly placing classes in chromosome and adding to population array
def generateFirstPopulation():
    population = []
    populationSize = 500

    for i in range(populationSize):
        chromosome = []
        for j in range(len(slots)):
            chromosome.append(day_room_time[random.randint(0, len(day_room_time)-1)])
        population.append(chromosome)
    return population

In [16]:
teacherSlots = defaultdict(list)
for i in range(0,len(slots)):
#     splits = slots[slot].split("$")
    teacherSlots[slots[i].teacher].append(i)
#     teacherSlots[splits[2]].append(splits[0])

In [17]:
print(teacherSlots)

defaultdict(<class 'list'>, {'T1': [0, 1, 2, 16, 17, 18, 32, 33, 34], 'T2': [3, 4, 19, 20, 35, 36], 'T3': [5, 6, 7, 21, 22, 23, 37, 38, 39], 'T4': [8, 9, 24, 25, 40, 41], 'T5': [10, 11, 12, 26, 27, 28, 42, 43, 44], 'T6': [13, 14, 15, 29, 30, 31, 45, 46, 47], 'T7': [48, 49, 63, 64, 78, 79, 80], 'T8': [50, 51, 65, 66, 81, 82], 'T9': [52, 53, 54, 67, 68, 83, 84, 85], 'T10': [55, 56, 57, 69, 70, 71, 86, 87, 88], 'T11': [58, 59, 60, 72, 73, 74, 89, 90], 'T12': [61, 62, 75, 76, 77, 91, 92, 93], 'T13': [94, 95, 96, 111, 112, 113, 129, 130], 'T14': [97, 98, 99, 114, 115, 116, 131, 132], 'T15': [100, 101, 117, 118, 119, 133, 134], 'T16': [102, 103, 104, 120, 121, 122, 135, 136], 'T17': [105, 106, 107, 123, 124, 125, 357, 358, 359], 'T18': [108, 109, 110, 126, 127, 128, 139, 140], 'T46': [137, 138, 189, 190, 191, 336, 337, 338], 'T19': [141, 142, 143, 159, 160, 161, 177, 178, 179], 'T20': [144, 145, 146, 162, 163, 164, 180, 181], 'T21': [147, 148, 149, 165, 166, 167, 182, 183], 'T22': [150, 151,

In [25]:
def calculateFitness(chromosome, checkSoftConstraint = True, checkTeacherPreference = True):
    
    softConstraintDelta = 1
    hardConstraintDelta = 5
    
    
    fitness = 5000
    
    for i in range(len(chromosome)):
        
        chromo = chromosome[i]
#         splits = chromo.split("_")
#         day = splits[0]
#         room = splits[1]
#         startTime = int(float(splits[2]))
    
        day = chromo.day
        room = chromo.room
        startTime = chromo.time
        
    
        #Teacher constraints
        
        s = slots[i]
#         splits = s.split("$")

        teacher = s.teacher
        duration = s.duration
        teaches = teacherSlots[teacher] #getting list of slots in which teacher teaches
        
        
        
        
        
        try:
            if teacherPreference[teacher] == 'm':
                if startTime + duration >= 13:
                    fitness -= softConstraintDelta
            elif teacherPreference[teacher] == 'e':
                if startTime + duration <= 13:
                    fitness -= softConstraintDelta
        except:
            pass
        
        for t in teaches:
            if t != i:#so that it does not test with itself
                
                #test data
                testChromo = chromosome[t]
#                 splits = testChromo.split("_")
#                 tday = splits[0]
#                 troom = splits[1]
#                 tstartTime = int(float(splits[2]))
                
                tday = testChromo.day
                troom = testChromo.room
                tstartTime = testChromo.time
            
                if tday == day:
                    if tstartTime == startTime: #teacher cant be at two places at the same time
                        fitness -= hardConstraintDelta
                    elif (startTime + duration == tstartTime) and checkSoftConstraint: #consecutive classes for teacher
                        fitness -= softConstraintDelta
        
        
        #Room constraints
        
        for j in range(len(chromosome)):
            if i != j:
                #test data
                testChromo = chromosome[j]
#                 splits = testChromo.split("_")
#                 tday = splits[0]
#                 troom = splits[1]
#                 tstartTime = int(float(splits[2]))
                
                tday = testChromo.day
                troom = testChromo.room
                tstartTime = testChromo.time
                
                
                if tday == day:
                    if troom == room:
                        if (startTime <= tstartTime < (startTime + duration)):
                            fitness -= hardConstraintDelta
                            
        
        #Friday Prayer Time 13-14
        if day == "Fri":
            if 13 <= startTime < 14:
                fitness -= hardConstraintDelta
    
    
    
    
            
    return fitness/5000   

In [26]:
def mutation(chromosome, levelOfRandomness = 0.2):
    
    for i in range(len(chromosome)):
        
        if random.uniform(0, 1) < levelOfRandomness:
            chromosome[i] = day_room_time[random.randint(0, len(day_room_time)-1)]
            
    return chromosome

In [27]:
def crossover(chromosome1, chromosome2):
    
    splits = random.sample(range(1, len(chromosome1)-1), 2)
    splits = sorted(splits)
    child1 = chromosome1[0:splits[0]]
    child2 = chromosome2[0:splits[0]]
    
    child1 = np.hstack([child1, chromosome2[splits[0]:splits[1]]])
    child2 = np.hstack([child2, chromosome1[splits[0]:splits[1]]])
    
#     child1.extend(chromosome2[splits[0]:splits[1]])
#     child2.extend(chromosome1[splits[0]:splits[1]])
    
    child1 = np.hstack([child1, chromosome1[splits[1]:]])
    child2 = np.hstack([child2, chromosome2[splits[1]:]])
    
#     child1.extend(chromosome1[splits[1]:])
#     child2.extend(chromosome2[splits[1]:])    
    
    return child1, child2

In [28]:
def nextGeneration(chromosome1, chromosome2):
    child1, child2 = crossover(chromosome1, chromosome2)
    
    
    child1 = mutation(child1)
    child2 = mutation(child2)
    
    
    return child1, child2
    

In [29]:
t1 = time.time()
population = generateFirstPopulation()
oldmaxFitness = -5000000
newmaxFitness = -5000000

generation = 0

while generation < 15:
    t2 = time.time()
    generation+=1
    print(generation)

    fitnesses = []
    for i in range(len(population)):
        fitnesses.append(calculateFitness(population[i]))
    fitnessinds = np.argsort(fitnesses)
    sortedFitness = np.array(fitnesses)[fitnessinds][::-1]
    sortedPopulation = np.array(population)[fitnessinds][::-1]

    bestPopulation = sortedPopulation[0:100]

    oldmaxFitness = newmaxFitness

    newmaxFitness = sortedFitness[0]

    print(sortedFitness[0:5])
    if (newmaxFitness >= 0.9):
        print(newmaxFitness)
        break


    nextPopulation = []
    for i in range(0, len(bestPopulation), 2):
        a, b = nextGeneration(bestPopulation[i], bestPopulation[i+1])
        nextPopulation.append(a)
        nextPopulation.append(b)
    nextPopulation.extend(bestPopulation[0:50])
    nextPopulation.extend(generateFirstPopulation()[:10])

    population = nextPopulation
    print("generation "+str(generation)+" took time: "+str(time.time()-t2))


print("Total time: "+str(time.time()-t1))
      

1
[0.7684 0.7666 0.7604 0.7562 0.7528]
generation 1 took time: 13.863706827163696
2
[0.7684 0.7666 0.764  0.7604 0.7562]
generation 2 took time: 5.23185396194458
3
[0.772  0.7684 0.7682 0.7666 0.764 ]
generation 3 took time: 5.41157603263855
4
[0.772  0.7684 0.7682 0.7666 0.7654]
generation 4 took time: 5.084594964981079
5
[0.772  0.7684 0.7682 0.7666 0.7654]
generation 5 took time: 5.391618728637695
6
[0.772  0.7686 0.7684 0.7682 0.7666]
generation 6 took time: 5.440612077713013
7
[0.7778 0.772  0.7686 0.7684 0.7682]
generation 7 took time: 5.295259952545166
8
[0.7778 0.772  0.7686 0.7684 0.7682]
generation 8 took time: 5.357497930526733
9
[0.7778 0.772  0.772  0.7686 0.7684]
generation 9 took time: 5.580457925796509
10
[0.7834 0.7778 0.777  0.772  0.772 ]
generation 10 took time: 5.335459232330322
11
[0.7834 0.7778 0.777  0.772  0.772 ]
generation 11 took time: 5.172194719314575
12
[0.7834 0.7778 0.777  0.774  0.772 ]
generation 12 took time: 5.455453872680664
13
[0.7852 0.7834 0.777

KeyboardInterrupt: 

In [33]:
c = bestPopulation[0]

In [34]:
calculateFitness(c)

0.7896

In [ ]:
a = ['a','f','e','b','d','c']
b=[1,6,5,2,4,3]

fitnessinds = np.argsort(b)
sa = list(a)[fitnessinds][:-1]
sb = array(b)[fitnessinds][::-1]

print(type(sa))
print(sa,sb)

In [ ]:
for item_a, item_b in zip(day_room_time, c):
    print(item_a, item_b)

In [ ]:
for r in range(roomCount):

        for j in range(3):

    #         print(day_room_time[(r*17)+j])
            val = c[(r*17)+j]
            if val!="":
    #             print(val)
                if teacher == val.split("_")[2]:
                    print(val.split("_")[2])
                    print(val)
                    print(day_room_time[(r*17)+j])

In [ ]:
for item_a, item_b in zip(day_room_time, c):
    print(item_a, item_b)

In [ ]:
child1, child2 = tools.cxTwoPoint(['1','2','3','4'], ['a','b','c','d'])


In [ ]:
print(child2)

In [ ]:
class slo:
    def __init__(self, a, b, c):
        self.a = a
        self.b = b
        self.c = c
    

In [ ]:
import time


In [ ]:
t1 = time.time()
s = slo('a','b','c')
for i in range(1000):
    
    val = s.a+s.b+s.c
#     print(val,)

print(time.time()-t1)

In [ ]:
t1 = time.time()
s = 'a$b$c'
for i in range(1000):
    
    splits = s.split('$')
    val = splits[0]+splits[1]+splits[2]
#     print(val,)

print(time.time()-t1)